In [2]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils
import play

In [3]:
from music21 import converter, instrument, note, chord, stream

In [4]:
songs = glob('Jazz/*.mid')

In [5]:
songs = songs[:3]

In [6]:
def get_notes():
    notes = []
    for file in songs:
        # converting .mid file to stream object
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                # if element is a note, extract pitch
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                # if element is a chord, append the normal form of the 
                # chord (a list of integers) to the list of notes. 
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [7]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    # Extract the unique pitches in the list of notes.
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [8]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [9]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'weights.best.music3.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [10]:
def train_network():
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = 200
    
    notes = get_notes()
    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    return model
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')

In [38]:
### Train the model 
train_network()

Notes processed
Vocab generated
Input and Output processed
Model created
Training in progress
Epoch 1/200
1184/1184 [==============================] - ETA: 1:22 - loss: 4.929 - ETA: 44s - loss: 4.907 - ETA: 32s - loss: 4.92 - ETA: 25s - loss: 4.88 - ETA: 21s - loss: 4.85 - ETA: 18s - loss: 4.80 - ETA: 16s - loss: 4.75 - ETA: 15s - loss: 4.76 - ETA: 13s - loss: 4.74 - ETA: 12s - loss: 4.72 - ETA: 11s - loss: 4.71 - ETA: 11s - loss: 4.68 - ETA: 10s - loss: 4.65 - ETA: 9s - loss: 4.6614 - ETA: 8s - loss: 4.648 - ETA: 8s - loss: 4.626 - ETA: 7s - loss: 4.628 - ETA: 7s - loss: 4.621 - ETA: 6s - loss: 4.611 - ETA: 6s - loss: 4.612 - ETA: 5s - loss: 4.616 - ETA: 5s - loss: 4.607 - ETA: 5s - loss: 4.615 - ETA: 4s - loss: 4.612 - ETA: 4s - loss: 4.602 - ETA: 3s - loss: 4.596 - ETA: 3s - loss: 4.582 - ETA: 3s - loss: 4.565 - ETA: 2s - loss: 4.553 - ETA: 2s - loss: 4.549 - ETA: 2s - loss: 4.548 - ETA: 1s - loss: 4.551 - ETA: 1s - loss: 4.550 - ETA: 1s - loss: 4.542 - ETA: 0s - loss: 4.537 - ETA: 

1184/1184 [==============================] - ETA: 10s - loss: 2.89 - ETA: 9s - loss: 2.7997 - ETA: 9s - loss: 2.769 - ETA: 9s - loss: 2.759 - ETA: 8s - loss: 2.625 - ETA: 8s - loss: 2.587 - ETA: 8s - loss: 2.562 - ETA: 8s - loss: 2.523 - ETA: 7s - loss: 2.525 - ETA: 7s - loss: 2.480 - ETA: 7s - loss: 2.441 - ETA: 6s - loss: 2.428 - ETA: 6s - loss: 2.447 - ETA: 6s - loss: 2.453 - ETA: 6s - loss: 2.448 - ETA: 5s - loss: 2.446 - ETA: 5s - loss: 2.426 - ETA: 5s - loss: 2.444 - ETA: 4s - loss: 2.432 - ETA: 4s - loss: 2.425 - ETA: 4s - loss: 2.436 - ETA: 4s - loss: 2.450 - ETA: 3s - loss: 2.460 - ETA: 3s - loss: 2.475 - ETA: 3s - loss: 2.478 - ETA: 3s - loss: 2.489 - ETA: 2s - loss: 2.493 - ETA: 2s - loss: 2.496 - ETA: 2s - loss: 2.498 - ETA: 1s - loss: 2.494 - ETA: 1s - loss: 2.508 - ETA: 1s - loss: 2.523 - ETA: 1s - loss: 2.532 - ETA: 0s - loss: 2.543 - ETA: 0s - loss: 2.554 - ETA: 0s - loss: 2.560 - 10s 9ms/step - loss: 2.5584
Epoch 18/200
1184/1184 [==============================] - ETA:

1184/1184 [==============================] - ETA: 9s - loss: 0.050 - ETA: 9s - loss: 0.077 - ETA: 9s - loss: 0.112 - ETA: 8s - loss: 0.117 - ETA: 8s - loss: 0.143 - ETA: 8s - loss: 0.129 - ETA: 8s - loss: 0.134 - ETA: 7s - loss: 0.131 - ETA: 7s - loss: 0.137 - ETA: 7s - loss: 0.133 - ETA: 7s - loss: 0.134 - ETA: 6s - loss: 0.144 - ETA: 6s - loss: 0.145 - ETA: 6s - loss: 0.149 - ETA: 5s - loss: 0.141 - ETA: 5s - loss: 0.146 - ETA: 5s - loss: 0.143 - ETA: 5s - loss: 0.144 - ETA: 4s - loss: 0.150 - ETA: 4s - loss: 0.149 - ETA: 4s - loss: 0.146 - ETA: 4s - loss: 0.144 - ETA: 3s - loss: 0.145 - ETA: 3s - loss: 0.144 - ETA: 3s - loss: 0.154 - ETA: 2s - loss: 0.161 - ETA: 2s - loss: 0.158 - ETA: 2s - loss: 0.156 - ETA: 2s - loss: 0.158 - ETA: 1s - loss: 0.157 - ETA: 1s - loss: 0.155 - ETA: 1s - loss: 0.156 - ETA: 1s - loss: 0.156 - ETA: 0s - loss: 0.160 - ETA: 0s - loss: 0.157 - ETA: 0s - loss: 0.156 - 10s 9ms/step - loss: 0.1557
Epoch 34/200
1184/1184 [==============================] - ETA: 

1184/1184 [==============================] - ETA: 9s - loss: 0.126 - ETA: 10s - loss: 0.06 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.06 - ETA: 9s - loss: 0.0533 - ETA: 9s - loss: 0.045 - ETA: 8s - loss: 0.040 - ETA: 8s - loss: 0.035 - ETA: 8s - loss: 0.034 - ETA: 7s - loss: 0.032 - ETA: 7s - loss: 0.029 - ETA: 7s - loss: 0.069 - ETA: 6s - loss: 0.065 - ETA: 6s - loss: 0.064 - ETA: 6s - loss: 0.061 - ETA: 6s - loss: 0.059 - ETA: 5s - loss: 0.057 - ETA: 5s - loss: 0.058 - ETA: 5s - loss: 0.055 - ETA: 4s - loss: 0.053 - ETA: 4s - loss: 0.051 - ETA: 4s - loss: 0.049 - ETA: 3s - loss: 0.049 - ETA: 3s - loss: 0.048 - ETA: 3s - loss: 0.059 - ETA: 3s - loss: 0.058 - ETA: 2s - loss: 0.056 - ETA: 2s - loss: 0.055 - ETA: 2s - loss: 0.053 - ETA: 1s - loss: 0.052 - ETA: 1s - loss: 0.052 - ETA: 1s - loss: 0.050 - ETA: 1s - loss: 0.049 - ETA: 0s - loss: 0.049 - ETA: 0s - loss: 0.048 - ETA: 0s - loss: 0.049 - 10s 9ms/step - loss: 0.0483
Epoch 50/200
1184/1184 [==============================] - ETA:

1184/1184 [==============================] - ETA: 9s - loss: 0.020 - ETA: 9s - loss: 0.011 - ETA: 9s - loss: 0.008 - ETA: 8s - loss: 0.007 - ETA: 8s - loss: 0.006 - ETA: 8s - loss: 0.005 - ETA: 8s - loss: 0.004 - ETA: 7s - loss: 0.004 - ETA: 7s - loss: 0.008 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.009 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 5s - loss: 0.009 - ETA: 5s - loss: 0.009 - ETA: 5s - loss: 0.008 - ETA: 4s - loss: 0.009 - ETA: 4s - loss: 0.009 - ETA: 4s - loss: 0.009 - ETA: 4s - loss: 0.009 - ETA: 3s - loss: 0.009 - ETA: 3s - loss: 0.008 - ETA: 3s - loss: 0.008 - ETA: 3s - loss: 0.009 - ETA: 2s - loss: 0.008 - ETA: 2s - loss: 0.008 - ETA: 2s - loss: 0.008 - ETA: 1s - loss: 0.008 - ETA: 1s - loss: 0.008 - ETA: 1s - loss: 0.008 - ETA: 1s - loss: 0.014 - ETA: 0s - loss: 0.014 - ETA: 0s - loss: 0.013 - ETA: 0s - loss: 0.013 - 10s 9ms/step - loss: 0.0136
Epoch 66/200
1184/1184 [==============================] - ETA: 

...

1184/1184 [==============================] - ETA: 10s - loss: 0.02 - ETA: 9s - loss: 0.0138 - ETA: 9s - loss: 0.127 - ETA: 9s - loss: 0.095 - ETA: 9s - loss: 0.095 - ETA: 8s - loss: 0.079 - ETA: 8s - loss: 0.068 - ETA: 8s - loss: 0.059 - ETA: 7s - loss: 0.053 - ETA: 7s - loss: 0.057 - ETA: 7s - loss: 0.053 - ETA: 7s - loss: 0.048 - ETA: 6s - loss: 0.045 - ETA: 6s - loss: 0.042 - ETA: 6s - loss: 0.039 - ETA: 5s - loss: 0.037 - ETA: 5s - loss: 0.035 - ETA: 5s - loss: 0.033 - ETA: 5s - loss: 0.031 - ETA: 4s - loss: 0.029 - ETA: 4s - loss: 0.028 - ETA: 4s - loss: 0.027 - ETA: 3s - loss: 0.026 - ETA: 3s - loss: 0.025 - ETA: 3s - loss: 0.024 - ETA: 3s - loss: 0.023 - ETA: 2s - loss: 0.023 - ETA: 2s - loss: 0.022 - ETA: 2s - loss: 0.021 - ETA: 1s - loss: 0.026 - ETA: 1s - loss: 0.025 - ETA: 1s - loss: 0.025 - ETA: 1s - loss: 0.024 - ETA: 0s - loss: 0.023 - ETA: 0s - loss: 0.023 - ETA: 0s - loss: 0.023 - 10s 9ms/step - loss: 0.0224
Epoch 130/200
1184/1184 [==============================] - ETA

1184/1184 [==============================] - ETA: 9s - loss: 4.3491e-0 - ETA: 9s - loss: 2.4667e-0 - ETA: 8s - loss: 3.4097e-0 - ETA: 8s - loss: 3.3492e-0 - ETA: 8s - loss: 0.0338    - ETA: 8s - loss: 0.035 - ETA: 7s - loss: 0.040 - ETA: 7s - loss: 0.057 - ETA: 7s - loss: 0.051 - ETA: 7s - loss: 0.045 - ETA: 6s - loss: 0.041 - ETA: 6s - loss: 0.038 - ETA: 6s - loss: 0.035 - ETA: 6s - loss: 0.033 - ETA: 5s - loss: 0.031 - ETA: 5s - loss: 0.029 - ETA: 5s - loss: 0.034 - ETA: 5s - loss: 0.032 - ETA: 4s - loss: 0.031 - ETA: 4s - loss: 0.030 - ETA: 4s - loss: 0.028 - ETA: 4s - loss: 0.034 - ETA: 3s - loss: 0.032 - ETA: 3s - loss: 0.032 - ETA: 3s - loss: 0.031 - ETA: 3s - loss: 0.030 - ETA: 2s - loss: 0.028 - ETA: 2s - loss: 0.032 - ETA: 2s - loss: 0.033 - ETA: 1s - loss: 0.032 - ETA: 1s - loss: 0.031 - ETA: 1s - loss: 0.030 - ETA: 1s - loss: 0.030 - ETA: 0s - loss: 0.029 - ETA: 0s - loss: 0.029 - ETA: 0s - loss: 0.029 - 10s 9ms/step - loss: 0.0311
Epoch 146/200
1184/1184 [==================

1184/1184 [==============================] - ETA: 9s - loss: 0.002 - ETA: 9s - loss: 0.001 - ETA: 9s - loss: 0.003 - ETA: 9s - loss: 0.061 - ETA: 8s - loss: 0.048 - ETA: 8s - loss: 0.040 - ETA: 8s - loss: 0.035 - ETA: 8s - loss: 0.030 - ETA: 7s - loss: 0.034 - ETA: 7s - loss: 0.034 - ETA: 7s - loss: 0.031 - ETA: 7s - loss: 0.028 - ETA: 6s - loss: 0.026 - ETA: 6s - loss: 0.024 - ETA: 6s - loss: 0.023 - ETA: 5s - loss: 0.026 - ETA: 5s - loss: 0.025 - ETA: 5s - loss: 0.023 - ETA: 5s - loss: 0.024 - ETA: 4s - loss: 0.022 - ETA: 4s - loss: 0.026 - ETA: 4s - loss: 0.026 - ETA: 3s - loss: 0.025 - ETA: 3s - loss: 0.024 - ETA: 3s - loss: 0.023 - ETA: 3s - loss: 0.022 - ETA: 2s - loss: 0.024 - ETA: 2s - loss: 0.027 - ETA: 2s - loss: 0.026 - ETA: 1s - loss: 0.026 - ETA: 1s - loss: 0.025 - ETA: 1s - loss: 0.024 - ETA: 1s - loss: 0.024 - ETA: 0s - loss: 0.031 - ETA: 0s - loss: 0.031 - ETA: 0s - loss: 0.030 - 10s 9ms/step - loss: 0.0312
Epoch 162/200
1184/1184 [==============================] - ETA:

1184/1184 [==============================] - ETA: 9s - loss: 0.105 - ETA: 9s - loss: 0.060 - ETA: 9s - loss: 0.040 - ETA: 9s - loss: 0.030 - ETA: 8s - loss: 0.024 - ETA: 8s - loss: 0.020 - ETA: 8s - loss: 0.017 - ETA: 7s - loss: 0.017 - ETA: 7s - loss: 0.016 - ETA: 7s - loss: 0.015 - ETA: 7s - loss: 0.013 - ETA: 6s - loss: 0.028 - ETA: 6s - loss: 0.050 - ETA: 6s - loss: 0.046 - ETA: 6s - loss: 0.043 - ETA: 5s - loss: 0.043 - ETA: 5s - loss: 0.040 - ETA: 5s - loss: 0.041 - ETA: 4s - loss: 0.039 - ETA: 4s - loss: 0.037 - ETA: 4s - loss: 0.035 - ETA: 4s - loss: 0.034 - ETA: 3s - loss: 0.047 - ETA: 3s - loss: 0.049 - ETA: 3s - loss: 0.047 - ETA: 3s - loss: 0.046 - ETA: 2s - loss: 0.044 - ETA: 2s - loss: 0.043 - ETA: 2s - loss: 0.041 - ETA: 1s - loss: 0.043 - ETA: 1s - loss: 0.042 - ETA: 1s - loss: 0.041 - ETA: 1s - loss: 0.049 - ETA: 0s - loss: 0.047 - ETA: 0s - loss: 0.046 - ETA: 0s - loss: 0.045 - 10s 9ms/step - loss: 0.0439
Epoch 178/200
1184/1184 [==============================] - ETA:

1184/1184 [==============================] - ETA: 11s - loss: 3.0547e- - ETA: 11s - loss: 9.6994e- - ETA: 11s - loss: 8.3953e- - ETA: 10s - loss: 1.2177e- - ETA: 10s - loss: 1.0728e- - ETA: 9s - loss: 9.5520e-05 - ETA: 9s - loss: 1.0620e-0 - ETA: 8s - loss: 9.8698e-0 - ETA: 8s - loss: 0.0012    - ETA: 8s - loss: 0.001 - ETA: 7s - loss: 0.001 - ETA: 7s - loss: 0.001 - ETA: 7s - loss: 0.001 - ETA: 6s - loss: 0.001 - ETA: 6s - loss: 0.001 - ETA: 6s - loss: 9.4522e-0 - ETA: 5s - loss: 8.9067e-0 - ETA: 5s - loss: 8.7165e-0 - ETA: 5s - loss: 8.8471e-0 - ETA: 4s - loss: 0.0011    - ETA: 4s - loss: 0.004 - ETA: 4s - loss: 0.004 - ETA: 4s - loss: 0.004 - ETA: 3s - loss: 0.004 - ETA: 3s - loss: 0.004 - ETA: 3s - loss: 0.004 - ETA: 2s - loss: 0.004 - ETA: 2s - loss: 0.004 - ETA: 2s - loss: 0.004 - ETA: 2s - loss: 0.004 - ETA: 1s - loss: 0.004 - ETA: 1s - loss: 0.004 - ETA: 1s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - ETA: 0s - loss: 0.004 - 11s 9ms/step - loss: 0.0040
Epoch 

In [11]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('weights.best.music3.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [12]:
def get_inputSequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)

In [13]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        
        # Predicted output is the argmax(P(h|D))
        index = np.argmax(prediction)
        # Mapping the predicted interger back to the corresponding note
        result = int_to_note[index]
        # Storing the predicted output
        prediction_output.append(result)

        pattern.append(index)
        # Next input to the model
        pattern = pattern[1:len(pattern)]

    print('Notes Generated...')
    return prediction_output

In [14]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output4.mid')

In [16]:
#### Generate a new jazz music 
generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........
Notes Generated...
Saving Output file as midi....


In [18]:
### Play the Jazz music
play.play_midi('test_output4.mid')

Music file test_output4.mid loaded!
